In [14]:
import os
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams, Distance
from config import COLLECTION_NAME, EMBED_RAW_PATH, EMBED_DIM, DATA_PREP_PATH

In [15]:
load_dotenv()

# Подключение к Qdrant
client = QdrantClient(
    url=os.getenv("QDRANT_URL"),
    api_key=os.getenv("QDRANT_API_KEY")
)

In [3]:
print(EMBED_RAW_PATH)

df = pq.read_table('embeddings.parquet', columns=['normalized_embed']).to_pandas()

data/raw/embeddings.parquet


In [4]:
x = df.iloc[0]
print(df.size)

7721749


In [5]:
x = x.tolist()

In [6]:
print(len(x))
print(f"Length of embedding: {len(x[0])}")

1
Length of embedding: 128


In [16]:
# Создаём коллекцию (если нет)
if not client.collection_exists(COLLECTION_NAME):
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=EMBED_DIM, distance=Distance.COSINE),
    )

In [17]:
parquet_file = pq.ParquetFile('embeddings.parquet')

In [20]:
df = pd.read_parquet('sessions_threshold_900.parquet')
item_ids_50m = pd.Series(df['item_ids'].explode()).unique()
item_ids_50m = set(item_ids_50m)

In [21]:
print(len(item_ids_50m))

886167


In [22]:
batch_size = 100
total_loaded = 0

for batch in parquet_file.iter_batches(batch_size=batch_size, columns=['item_id', 'normalized_embed']):
    df_batch = batch.to_pandas()
    
    # Фильтруем только нужные item_id
    mask = df_batch['item_id'].isin(item_ids_50m)
    df_filtered = df_batch[mask]
    
    if len(df_filtered) == 0:
        continue  # Пропускаем пустые батчи
    
    # Формируем точки для Qdrant
    points = []
    for _, row in df_filtered.iterrows():
        vector = np.array(row['normalized_embed'], dtype=np.float32).tolist()
        points.append(
            PointStruct(
                id=int(row['item_id']),  # Qdrant требует int или str
                vector=vector,
                payload={}  # Можно добавить метаданные позже
            )
        )
    
    # Загружаем в Qdrant
    client.upsert(collection_name=COLLECTION_NAME, points=points)
    total_loaded += len(points)
    print(f"Загружено: {total_loaded:,} / {len(item_ids_50m):,} треков")

print(f"Готово! Загружено {total_loaded:,} эмбеддингов.")

Загружено: 7 / 886,167 треков
Загружено: 27 / 886,167 треков
Загружено: 35 / 886,167 треков
Загружено: 48 / 886,167 треков
Загружено: 58 / 886,167 треков
Загружено: 76 / 886,167 треков
Загружено: 85 / 886,167 треков
Загружено: 96 / 886,167 треков
Загружено: 105 / 886,167 треков
Загружено: 122 / 886,167 треков
Загружено: 134 / 886,167 треков
Загружено: 146 / 886,167 треков
Загружено: 162 / 886,167 треков
Загружено: 176 / 886,167 треков
Загружено: 185 / 886,167 треков
Загружено: 195 / 886,167 треков
Загружено: 204 / 886,167 треков
Test run ended
Готово! Загружено 204 эмбеддингов.
